# Pycuda-euler 

GPU assembler using PySpark for distributed computing

In [1]:
from pyspark import SparkContext

In [2]:
import eulercuda as ec

/anaconda/envs/python34/lib/python3.4/site-packages/graph_tool/draw/cairo_draw.py:1468: RuntimeWarning: Error importing Gtk module: No module named 'gi'; GTK+ drawing will not work.
  warnings.warn(msg, RuntimeWarning)


In [3]:
import pycuda.driver
import pycuda.autoinit

In [ ]:
# sc = SparkContext(master='local[*]')

In [4]:
sc.defaultParallelism

8

In [5]:
sc.defaultMinPartitions

2

In [6]:
complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}

Load data and check how many ways Spark/HDFS slices it up...

In [82]:
raw_data = sc.textFile('hdfs://localhost:9000/Genome/Ba10k.sim1.fq')
raw_data.getNumPartitions()
# raw_data.count()

2

Data will either be in "fasta" or "fastq" format, meaning it will have more than just the DNA letters in it. We'll have to clean that up. 

**fastq parsing**

In [83]:
data = raw_data.filter(lambda x: x[0] in ['A','C','G','T'])
dataLength = len(data.take(1)[0])
dataCount = data.count() // data.getNumPartitions()


The k value determines how long the dna substrings are. K value is always an odd number and is usually slightly more than 1/2 the size of the read length (read length is number of DNA characters on one line in the data file).

It should be noted that although k value is the often-seen metric in the literature, Mahmood's GPU-Euler called out an l ("ell") value, because l is one more than k and it is that lth character that forms the edge between two kmer nodes.

In [84]:
k = 17
lmerLength = 18

In [85]:
%time hail_mary = data.mapPartitions(lambda x: ec.assemble2(k, buffer=x, readLength = dataLength,readCount=dataCount)).collect()

CPU times: user 15.1 ms, sys: 19.2 ms, total: 34.3 ms
Wall time: 1min 10s


**fasta parsing**

In [ ]:
# placeholder cell

I would like to think I could simply replace eulercuda.py's assemble2() by replicating function calls here, but it probably won't be that easy. 

In [39]:
data.take(2)

['AAGTTGCGCTAGGGTTAAACTCGGCTAACTCGATTAACATCAGCCGTTTGGTGGCGCAGATTTGCTACTA',
 'CGCCGCCATGCCGACCATCCCTTTCATCCCCGTACCAGACACGCTGACCATTGCCATGTTATTCAGATTG']

In [40]:
readLength = len(data.take(1)[0])
readLength

70

In [41]:
readCount = data.count() * readLength
readCount

70000

The original C used a buffer that was one *really* long string. This could be... problematic.

In [42]:
buffer = data.mapPartitions(lambda x: ''.join(x))

In [43]:
buffer

PythonRDD[18] at RDD at PythonRDD.scala:48

In [44]:
baseCount = buffer.count()

In [45]:
baseCount

70000

In [46]:
buffer.getNumPartitions()

2

In [58]:
glom = buffer.repartition(1)
glom.getNumPartitions()

1

In [16]:
    evList = []
    eeList = []
    edgeCount = 0
    levEdgeList = []
    entEdgeList = []
    edgeCount = 0
    vertexCount = 0

#### Note: 
**Important distinction:** 
if we have a one-argument function:

    def myFunc(x):

we can call it like this:

    myNewRDD = rdd.map(myFunc)

and rdd will be passed to myFunc as parameter x. **But** if we have multiple parameters, we handle it with lambda:

    def myOtherFunc(x, y, z):
    
    newRDD = rdd.map(lambda j: myOtherFunc(j, y, z))


In [59]:
# use lambda form above. Each buffer partition will be passed to constructDebruijnGraph
partitionReadCount = readCount // buffer.getNumPartitions()
rdd_graph = glom.mapPartitions(lambda x: ec.constructDebruijnGraph(x, partitionReadCount, readLength, lmerLength, evList, eeList, levEdgeList, entEdgeList, readCount))
                                
et_graph= rdd_graph.mapPartitions(lambda x: ec.findEulerTour(x[1], x[0], x[2], x[3], x[5], x[4], lmerLength, "read1_spark.txt"))


In [62]:
# rdd_Graph.getNumPartitions()
# evList.tolist()
#%time 
# rdd_Graph.collect()
# et_graph.collect()

operation above gets to setup_edges_device before erroring out. 

Based on console messages we're not calculating edges properly

Maybe try calling functions one by one?

In [ ]:
ø

In [24]:
#findEulerTour(evList, eeList, levEdgeList, entEdgeList, edgeCount, vertexCount, lmerLength, outfile)
rdd_tour = rdd_Graph.mapPartitions(lambda x: ec.findEulerTour(x[1], x[0], x[2], x[3], x[5], x[4], lmerLength, "ba_spark.txt"))

In [61]:
#rdd_tour.collect()

_constructDebruijnGraph_ functions:

    vals = readLmersKmersCuda(readBuffer, readLength, readCount, lmerLength, h_lmerKeys, h_lmerValues,
                              lmerCount, h_kmerKeys, h_kmerValues, kmerCount, numReads)
    results = gh.create_hash_table_device(h_kmerKeys, h_kmerValues, kmerCount, d_TK, d_TV, tableLength,
                                          d_bucketSize, bucketCount)

    tableLength = results[0]
    d_bucketSize = results[1]
    bucketCount = results[2]
    d_TK = results[3]       # kmer keys in sorted buckets
    d_TV = results[4]       # kmer values in sorted buckets

    d_ev, d_ee, d_levEdge, d_entEdge, kmerCount, edgeCount = \
        db.construct_debruijn_graph_device(h_lmerKeys, h_lmerValues, lmerCount, h_kmerKeys, kmerCount, lmerLength,
        d_TK, d_TV, d_bucketSize, bucketCount, d_ev, d_levEdge, d_entEdge, d_ee, readLength, readCount)


In [63]:
# vals =  buffer.mapPartitions(lambda x: readLmersKmersCuda(x, readLength, readCount, lmerLength, h_lmerKeys, h_lmerValues,
#                               lmerCount, h_kmerKeys, h_kmerValues, kmerCount, numReads))

hail_mary = data.mapPartitions(lambda x: ec.assemble2(k, x)).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 23.0 failed 1 times, most recent failure: Lost task 1.0 in stage 23.0 (TID 42, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/Cellar/apache-spark/2.0.1/libexec/python/pyspark/rdd.py", line 317, in func
    return f(iterator)
  File "<ipython-input-63-18f7a3ee3b93>", line 4, in <la